## Automated Monitoring - Deployment Monitoring Integration¶

Verta provides a extensible [model monitoring framework](https://docs.verta.ai/verta/monitoring) that allows the user to fully define and configure what data to monitor and how to monitor it.

This is an example to showcase how to use automated monitoring when you deploy models in Verta. Learn more about [automated monitoring for Verta endpoints](https://docs.verta.ai/verta/monitoring/guides/automated-monitored-entities).

The Verta client offers the ability to profile your training data and upload their distributions using log_training_data_profile()to monitor live inputs and outputs against expected thresholds and track missing data.

## 0. Imports¶

In [ ]:
from  sklearn import  datasets
iris=datasets.load_iris()

## 0.1 Verta import and setup¶

In [ ]:
# restart your notebook if prompted on Colab
try:
    import verta
except ImportError:
    !pip install verta

In [ ]:
# import os
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 
# os.environ['VERTA_HOST']

from verta import Client
client = Client(os.environ['VERTA_HOST'])

## 1. Model Training¶

### 1.1 Load training data¶

In [ ]:
x=iris.data
y=iris.target

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.5)

In [ ]:
import pandas as pd
X_train = pd.DataFrame(x_train, columns=iris.feature_names)
Y_train = pd.Series(y_train, name="class")

### 1.2 Fit Model¶

In [ ]:
from sklearn import tree
classifier=tree.DecisionTreeClassifier()

In [ ]:
classifier.fit(x_train,y_train)

## 2. Register model for deployment¶

In [ ]:
registered_model = client.get_or_create_registered_model(
    name="monitoring-iris-example-meeta")

### 2.1 Create model version¶

In [ ]:
from verta.environment import Python
from verta.utils import ModelAPI

model_version = registered_model.create_standard_model_from_sklearn(
    classifier,
    environment=Python(requirements=["sklearn"]),
    model_api=ModelAPI(X_train, Y_train),
    name="v1"
)

In [ ]:
model_version

### 2.2 Profile training data

In [ ]:
model_version.log_training_data_profile(X_train,Y_train)

## 3. Deploy model and start monitoring¶

In [ ]:
iris_endpoint = client.get_or_create_endpoint("monitoring-iris-example-meeta")
iris_endpoint.update(model_version, wait=True)

### 3.1 Make predictions¶

In [ ]:
deployed_model = iris_endpoint.get_deployed_model()
deployed_model.predict([x_train[0]])

In [ ]:
x_train.shape[0]

### 3.2 Run a batch prediction 

In [ ]:
deployed_model = iris_endpoint.get_deployed_model()
print(deployed_model)
for i in range(x_train.shape[0]):
    print(deployed_model.predict([x_train[i]]))
    
    if i == 50:
        break